[![gammapy](https://img.shields.io/badge/powered%20by-gammapy-orange.svg?style=flat)](https://gammapy.org/)

# Gammapy Part II: Spectral Analysis 
**Tutors:** Rubens Costa Jr.

This hands-on tutorial gives an introduction and overview of [Gammapy](https://gammapy.org/). 
In the first part of the this tutorial we learned about the basic data structures in Gammapy using [Third Fermi-LAT Catalog of High-Energy Sources (3FHL) catalog](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/3FHL/), in this second part we will perform a spectral analysis using simulated CTA data.

## Preface
We recommend to follow this tutorial by **executing the code cells on your local machine**, along with the tutor. The estimated time for this part of the tutorial is ~30 minutes.

We're happy to receive any **feedback or questions** on the tutorial via mail to *rubensjrcosta@gmail.com*.


___

<a id='setup'></a>
🔝 [Back to Top](#intro)<br>
## 2. Setup 

**Important**: to run this tutorial the environment variable `GAMMAPY_DATA` must be defined and point to the directory on your machine where the datasets needed are placed. To check whether your setup is correct you can execute the following cell:



In [1]:
import os # Miscellaneous operating system interfaces

path = os.path.expandvars("$GAMMAPY_DATA")

if not os.path.exists(path):
    raise Exception("gammapy-data repository not found!")
else:
    print("Great your setup is correct!")

Great your setup is correct!


In case you encounter an error, you can un-comment and execute the following cell to continue. But we recommend to set up your environment correctly as described in [getting started](https://docs.gammapy.org/dev/getting-started/index.html#download-tutorials) after you are done with this notebook.

In [ ]:
# os.environ['GAMMAPY_DATA'] = os.path.join(os.getcwd(), '..')

Now we can continue with the usual IPython notebooks and Python imports:

In [2]:
# Display figures directly inline
%matplotlib inline 

import matplotlib.pyplot as plt # A collection of command style functions

In [3]:
# I'd like to ignore some deprecation warnings by matplotlib
# This in general not advised, but for this specific notebook
from matplotlib import MatplotlibDeprecationWarning # A class for issuing deprecation warnings 

import warnings # Warning control
warnings.filterwarnings(
    "ignore", category=MatplotlibDeprecationWarning
)

In [4]:
import gammapy
from astropy import units as u
import numpy as np
from astropy.io import ascii
import collections
import sys, os
import matplotlib.pyplot as plt
from gammapy.makers import SpectrumDatasetMaker, SafeMaskMaker, ReflectedRegionsBackgroundMaker
from gammapy.modeling import Fit
from gammapy.data import Observation, Observations
from gammapy.datasets import SpectrumDatasetOnOff, SpectrumDataset, Datasets
from gammapy.irf import load_cta_irfs
from gammapy.maps import MapAxis, RegionGeom, WcsNDMap, WcsGeom

from gammapy.modeling.models import (
    EBLAbsorptionNormSpectralModel,
    Models,
    PowerLawSpectralModel,
    SkyModel,
    ExpCutoffPowerLawSpectralModel
)

from gammapy.irf import EffectiveAreaTable2D

from numpy.random import RandomState

from scipy.stats import chi2, norm

from gammapy.estimators import FluxPointsEstimator
from gammapy.estimators import FluxPoints
from gammapy.datasets import FluxPointsDataset

# astropy imports
from astropy.coordinates import SkyCoord, Angle
from astropy import units as u
from astropy.io import fits
from astropy.table import Table, Column

from gammapy.estimators import SensitivityEstimator

# astropy affiliated packages imports
from regions import CircleSkyRegion

from gammapy.stats import WStatCountsStatistic
from gammapy.stats import CashCountsStatistic
from scipy.stats import sem
from gammapy.maps import Map

import math

import pandas as pd

In [5]:
# Define simulation parameters 
livetimes = [2.21] * u.h # Livetime exposure of the simulated observation
sites = ['North'] # CTA North
irftime = '0.5h' #  Observation times

In [6]:
# Analysis parameters
on_region_radius = Angle("0.11 deg")
alpha = 0.2  # acceptance_off=1./alpha) # Relative background efficiency in the off region
emin = 1 * u.GeV
emax = 100. * u.TeV

In [7]:
# We simulate each observation n_obs times, to randomize the renortets:
n_obs = 1

In [8]:
df_simulated = pd.read_csv('bllac_norte_1x/bllac_norte_1x.txt', sep='  ', engine='python')
display(df_simulated)

,Source_Name,RA_pointing,DEC_pointing,RA,DEC,Redshift,Luminosity,Index,Amplitude,IRF_zen,Offset
0,73,263.388769,10.531347,262.7413,9.4659,0.5974,41.4399,1.8942,7.376000e-12,20,1.241670
1,107,193.529833,24.267100,191.7998,25.4312,0.1695,42.3985,1.7332,3.003200e-10,20,1.954334
2,150,185.251999,11.035411,182.7424,10.5479,0.2588,41.6568,1.5858,5.644100e-12,20,2.512943
3,231,185.251999,11.035411,183.1533,9.3930,0.9753,41.7015,1.5463,2.220900e-13,20,2.638790
4,656,284.450004,18.987378,286.3909,19.0569,0.3053,42.5607,1.9342,6.487500e-10,20,1.836216
...,...,...,...,...,...,...,...,...,...,...,...
11959,1306436,299.994430,43.648297,304.6044,44.2211,0.6020,41.8675,1.9608,3.275900e-11,20,3.368360
11960,1306565,185.251999,11.035411,187.0346,9.4757,0.1000,41.7215,1.4820,1.860700e-11,20,2.347216
11961,1306700,304.273901,48.374328,308.9487,48.8046,0.6514,41.4638,1.4280,1.068100e-13,20,3.121425
11962,1306871,185.251999,11.035411,181.7325,10.7188,0.2964,42.3393,1.6627,4.073700e-11,20,3.470716


In [9]:
selected_obs = df_simulated.sample(n = 2).reset_index(drop=True)
selected_obs

,Source_Name,RA_pointing,DEC_pointing,RA,DEC,Redshift,Luminosity,Index,Amplitude,IRF_zen,Offset
0,402592,251.315112,10.620350,248.2407,9.1647,0.1919,40.8288,1.8182,1.298100e-11,20,3.360259
1,628502,299.994430,43.648297,301.7109,41.6038,0.4461,40.8478,1.8338,2.293000e-12,20,2.402995


In [10]:
selected_obs.to_string('bllac_norte_1x/selected_obs.txt', index=False)

In [11]:
source_name, ra_pointing, dec_pointing, ra, dec, red, luminosity, index, amplitude, IRF_zen, offset = np.loadtxt('bllac_norte_1x/selected_obs.txt', skiprows=1, unpack=True)

In [12]:
VERBOSE = True

In [13]:
fitbackend = 'minuit'
optimize_opts = {'tol': 0.1, 'strategy': 1, 'print_level': 3}

In [14]:
output = {'Source_Name': [],
          'RA_pointing': [],
          'DEC_pointing': [],
          'RAJ2000': [],
          'DEJ2000': [],
          'Redshift': [],
          'Amplitude': [],
          'Index': [],
          'Luminosity': [],
          'Offset': [],
          'Site': [],
          'Irf_Zen_pointing': [],
          'Irf_Time': [],
          'Livetime_pointing': [],
          'Emin': [],
          'Ethreshold': [],
          'AboveEthFlag': [],
          'Model_Name': [],
          'N_sims' : [],
          'N_failed' : [],
          'Bkg': [],
          'Significance': [],
          'Energy0': [],
          'Energy1': [],
          'eref0': [],
          'eref1': [],
          'eref2': [],
          'eref3': [],
          'eref4': [],
          'eref5': [],
          'eref6': [],
          'eref7': [],
          'eref8': [],
          'eref9': [],
          'eref10': [],
          'eref11': [],
          'emin0': [],
          'emin1': [],
          'emin2': [],
          'emin3': [],
          'emin4': [],
          'emin5': [],
          'emin6': [],
          'emin7': [],
          'emin8': [],
          'emin9': [],
          'emin10': [],
          'emin11': [],
          'emax0': [],
          'emax1': [],
          'emax2': [],
          'emax3': [],
          'emax4': [],
          'emax5': [],
          'emax6': [],
          'emax7': [],
          'emax8': [],
          'emax9': [],
          'emax10': [],
          'emax11': [],
          'dnde0': [],
          'dnde1': [],
          'dnde2': [],
          'dnde3': [],
          'dnde4': [],
          'dnde5': [],
          'dnde6': [],
          'dnde7': [],
          'dnde8': [],
          'dnde9': [],
          'dnde10': [],
          'dnde11': [],
          'dnde_err0': [],
          'dnde_err1': [],
          'dnde_err2': [],
          'dnde_err3': [],
          'dnde_err4': [],
          'dnde_err5': [],
          'dnde_err6': [],
          'dnde_err7': [],
          'dnde_err8': [],
          'dnde_err9': [],
          'dnde_err10': [],
          'dnde_err11': [],
          'amplitude_fit': [],
          'amplitude_err_fit': [],
          'index_fit': [],
          'index_err_fit': [],
          'reference_fit': [],
          'int_sens': []

      }

In [17]:
#for i in range(len(name)):
for i in range(len(selected_obs)):
    print(f"""#################### source_name = {source_name[i]} ####################""")
    
    # Define Target Region
    # source position
    src_position = SkyCoord(
        ra[i], 
        dec[i], 
        unit="deg", 
        frame="icrs"
    )
    
    # observation region
    on_region = CircleSkyRegion(
        center=src_position, 
        radius=on_region_radius
    )

    
    # Define simulation parameters
    pointing = SkyCoord(
         ra_pointing[i], 
         dec_pointing[i], 
         unit="deg", 
         frame="icrs"
     )
    
    # Define spectral model - a simple Power Law in this case
    specmodel = PowerLawSpectralModel(
        amplitude=amplitude[i] * u.Unit('cm^-2 s^-1 MeV^-1'),
        reference=1 * u.Unit('MeV'),
        index=index[i]
    )
    
    # EBL absorbption spectral model
    absorption = EBLAbsorptionNormSpectralModel.read_builtin(
        "finke", # Available model in gammapy-data:{'franceschini', 'dominguez', 'finke'}
        redshift=red[i]
    )
    
    absspecmodel = specmodel * absorption

    skymodel = SkyModel(
        spectral_model=absspecmodel, 
        name="crab"
    )
    
    obs1 = Observation.create(
        pointing=pointing, 
        livetime=livetimes, # Livetime exposure of the simulated observation
        irfs=load_cta_irfs('/home/bornagain/Documents/GitHub/gammapy/gammapy-notebooks/0.20.1/tutorials/data/caldb/data/cta/prod3b-v2/bcf/North_z'+str(int(IRF_zen[i]))+'_0.5h/irf_file.fits')
    )
    obs = [obs1]

    AboveEthFlag = True
    
    # reconstructed energy
    energy_reco = MapAxis.from_energy_bounds(
        emin, 
        emax, 
        nbin=5, 
        per_decade=True, 
        name="energy"
    )
    
    # true energy should be wider than reconstructed energy
    energy_true = MapAxis.from_energy_bounds(
        0.3*emin, 
        3*emax, 
        nbin=8, 
        per_decade=True, 
        name="energy_true"
    )

    significances = []

    # Make the SpectrumDataset from observation parameters
    geom = RegionGeom.create(
        region=on_region, 
        axes=[energy_reco]
    )
    
    # Make the SpectrumDataset
    dataset_empty = SpectrumDataset.create(
        geom=geom, 
        energy_axis_true=energy_true, 
        name=int(source_name[i])
    )
    
    # The irfs and background are computed at a single fixed offset
    maker = SpectrumDatasetMaker(
        # Apply containment correction for point sources and circular on regions
        containment_correction=True, 
        # Selecting which maps to make
        selection=["edisp", "background", "exposure"]
    )
    
    # Make safe data range mask for a given observation
    safe_maker = SafeMaskMaker(methods=["bkg-peak"])

    n_sims = 0
    n_failed = 0
    significance=[]
    eref0 = []
    eref1 = []
    eref2 = []
    eref3 = []
    eref4 = []
    eref5 = []
    eref6 = []
    eref7 = []
    eref8 = []
    eref9 = []
    eref10 = []
    eref11 = []
    emin0 = []
    emin1 = []
    emin2 = []
    emin3 = []
    emin4 = []
    emin5 = []
    emin6 = []
    emin7 = []
    emin8 = []
    emin9 = []
    emin10 = []
    emin11 = []
    emax0 = []
    emax1 = []
    emax2 = []
    emax3 = []
    emax4 = []
    emax5 = []
    emax6 = []
    emax7 = []
    emax8 = []
    emax9 = []
    emax10 = []
    emax11 = []
    dnde0 = []
    dnde1 = []
    dnde2 = []
    dnde3 = []
    dnde4 = []
    dnde5 = []
    dnde6 = []
    dnde7 = []
    dnde8 = []
    dnde9 = []
    dnde10 = []
    dnde11 = []
    dnde_err0 = []
    dnde_err1 = []
    dnde_err2 = []
    dnde_err3 = []
    dnde_err4 = []
    dnde_err5 = []
    dnde_err6 = []
    dnde_err7 = []
    dnde_err8 = []
    dnde_err9 = []
    dnde_err10 = []
    dnde_err11 = []
    index_fit= []
    amplitude_fit = []
    amplitude_err_fit = []
    index_fit = []
    index_err_fit = []
    reference_fit = []
    datasets = Datasets()
    
    while n_sims < n_obs:
        if VERBOSE:
            print(f"""#################### n_sims = {n_sims+1} ####################""")
            for obs_id in obs:
                
                dataset = maker.run(
                    dataset_empty, 
                    obs_id
                )
                
                dataset = safe_maker.run(
                    dataset, 
                    obs_id
                )
                
                # Energy range maps defined by the mask_safe only
                low_energies_safe = dataset.energy_range_safe[0]
                
                # Energy threshold
                energy_threshold = low_energies_safe.get_by_coord({'skycoord': src_position})[0] * low_energies_safe.unit
                
                # Create spectrum dataseton off from another dataset
                dataset_onoff = SpectrumDatasetOnOff.from_spectrum_dataset(
                    dataset=dataset, 
                    acceptance=1, # Relative background efficiency in the on region
                    acceptance_off=1./alpha
                ) # Relative background efficiency in the off region (alpha = 0.2)
                
                # Add the model on the dataset
                skymodel_fit = skymodel.copy()
                dataset_onoff.models = skymodel_fit

                # Simulate fake counts (on and off) for the current model and reduced IRFs
                try:
                    dataset_onoff.fake(
                        random_state='random-seed', # Defines random number generator initialisation.
                        npred_background=dataset.npred_background() # Predicted background counts
                    )
                except ValueError as e:
                    print(f'DEBUG JPL: In fake: Caught {e}')
                    n_failed += 1
                    continue
                
                # append dataset_onoff to the datasets
                datasets.append(dataset_onoff)
            
            #  stack all dataset
            dataset_stacked = Datasets(datasets).stack_reduce(name=int(source_name[i]))
            
            # copy skymodel to fit it
            skymodel_fit = skymodel.copy()
            
            # loads the model into the staked dataset
            dataset_stacked.models = skymodel_fit

            skymodel_fit.parameters["reference"].quantity = 0.2 * u.TeV
            skymodel_fit.parameters["amplitude"].value *= (1.+(2.*np.random.rand(1)-1.)/10.)
            skymodel_fit.parameters["index"].value *= (1.+(2.*np.random.rand(1)-1.)/10.)

            stacked_fit = Fit(backend=fitbackend, optimize_opts=optimize_opts)
           
            renortet_stacked = stacked_fit.run([dataset_stacked])
            print(renortet_stacked.parameters["amplitude"].quantity.value)

            amplitude_fit.append(renortet_stacked.parameters["amplitude"].quantity.value)
            amplitude_err_fit.append(renortet_stacked.parameters["amplitude"].error)
            index_fit.append(renortet_stacked.parameters["index"].quantity.value)
            index_err_fit.append(renortet_stacked.parameters["index"].error)
            reference_fit.append(renortet_stacked.parameters["reference"].quantity.value)

            spectral_model = PowerLawSpectralModel(
                amplitude=amplitude[i] * u.Unit('cm^-2 s^-1 MeV^-1'),
                reference=1 * u.Unit('MeV'),
                index=index[i]
            )

            absorption_ebl = EBLAbsorptionNormSpectralModel.read_builtin("finke", redshift=red[i])
            absspecmodel_ebl = spectral_model * absorption_ebl
            
            model = SkyModel(spectral_model=absspecmodel_ebl, name="crab")

            dataset_stacked.models = [model]
            
            
            # Estimates flux points for a given list of datasets, energies and spectral model
            
            energy_edges = MapAxis.from_energy_bounds("0.1 TeV", "30 TeV", nbin=12).edges
            fpe = FluxPointsEstimator(energy_edges=energy_edges, source="crab", selection_optional="all")
            flux_points = fpe.run(datasets=dataset_stacked)
            
            print(flux_points.to_table(sed_type="dnde", formatted=True))
            print(flux_points["dnde"].data)
            
            # Saves the flux points
            file = open("/home/bornagain/Documents/GitHub/gammapy/gammapy-notebooks/0.20.1/presentations/gammapy/results/flux_point_src"+str(int(source_name[i]))+".txt","+w")
            file.write('E_ref (TeV) E_max (TeV) E_min (TeV) dnde (cm-2 s-1 MeV-1) dnde_err \r\n')

            for j in range(len(flux_points["energy_max"].value)):
                if flux_points["dnde"].data[j] > 0:
                    file.write(str(flux_points["energy_ref"].value[j]) + '  ' + str(flux_points["energy_max"].value[j]) + '  ' + str(flux_points["energy_min"].value[j]) + '  ' + str(flux_points["dnde"].data[j])[1:-1] + '  ' + str(flux_points["dnde_err"].data[j])[1:-1] + '\n')

            eref0.append(flux_points["energy_ref"].data[0])
            eref1.append(flux_points["energy_ref"].data[1])
            eref2.append(flux_points["energy_ref"].data[2])
            eref3.append(flux_points["energy_ref"].data[3])
            eref4.append(flux_points["energy_ref"].data[4])
            eref5.append(flux_points["energy_ref"].data[5])
            eref6.append(flux_points["energy_ref"].data[6])
            eref7.append(flux_points["energy_ref"].data[7])
            eref8.append(flux_points["energy_ref"].data[8])
            eref9.append(flux_points["energy_ref"].data[9])
            eref10.append(flux_points["energy_ref"].data[10])
            eref11.append(flux_points["energy_ref"].data[11])
            emin0.append(flux_points["energy_min"].data[0])
            emin1.append(flux_points["energy_min"].data[1])
            emin2.append(flux_points["energy_min"].data[2])
            emin3.append(flux_points["energy_min"].data[3])
            emin4.append(flux_points["energy_min"].data[4])
            emin5.append(flux_points["energy_min"].data[5])
            emin6.append(flux_points["energy_min"].data[6])
            emin7.append(flux_points["energy_min"].data[7])
            emin8.append(flux_points["energy_min"].data[8])
            emin9.append(flux_points["energy_min"].data[9])
            emin10.append(flux_points["energy_min"].data[10])
            emin11.append(flux_points["energy_min"].data[11])
            emax0.append(flux_points["energy_max"].data[0])
            emax1.append(flux_points["energy_max"].data[1])
            emax2.append(flux_points["energy_max"].data[2])
            emax3.append(flux_points["energy_max"].data[3])
            emax4.append(flux_points["energy_max"].data[4])
            emax5.append(flux_points["energy_max"].data[5])
            emax6.append(flux_points["energy_max"].data[6])
            emax7.append(flux_points["energy_max"].data[7])
            emax8.append(flux_points["energy_max"].data[8])
            emax9.append(flux_points["energy_max"].data[9])
            emax10.append(flux_points["energy_max"].data[10])
            emax11.append(flux_points["energy_max"].data[11])
            dnde0.append(flux_points["dnde"].data[0])
            dnde1.append(flux_points["dnde"].data[1])
            dnde2.append(flux_points["dnde"].data[2])
            dnde3.append(flux_points["dnde"].data[3])
            dnde4.append(flux_points["dnde"].data[4])
            dnde5.append(flux_points["dnde"].data[5])
            dnde6.append(flux_points["dnde"].data[6])
            dnde7.append(flux_points["dnde"].data[7])
            dnde8.append(flux_points["dnde"].data[8])
            dnde9.append(flux_points["dnde"].data[9])
            dnde10.append(flux_points["dnde"].data[10])
            dnde11.append(flux_points["dnde"].data[11])
            dnde_err0.append(flux_points["dnde_err"].data[0])
            dnde_err1.append(flux_points["dnde_err"].data[1])
            dnde_err2.append(flux_points["dnde_err"].data[2])
            dnde_err3.append(flux_points["dnde_err"].data[3])
            dnde_err4.append(flux_points["dnde_err"].data[4])
            dnde_err5.append(flux_points["dnde_err"].data[5])
            dnde_err6.append(flux_points["dnde_err"].data[6])
            dnde_err7.append(flux_points["dnde_err"].data[7])
            dnde_err8.append(flux_points["dnde_err"].data[8])
            dnde_err9.append(flux_points["dnde_err"].data[9])
            dnde_err10.append(flux_points["dnde_err"].data[10])
            dnde_err11.append(flux_points["dnde_err"].data[11])
            
            # Extract analysis statistics
            significancee = WStatCountsStatistic(n_on=sum(dataset_stacked.counts.data), n_off=sum(dataset_stacked.counts_off.data), alpha=0.2).sqrt_ts
            significance.append(significancee)
 
            n_sims += 1
            print(np.median(significance))

    output['Source_Name'].append(int(source_name[i]))
    output['RA_pointing'].append(pointing.ra.value)
    output['DEC_pointing'].append(pointing.dec.value)
    output['RAJ2000'].append(src_position.ra.value)
    output['DEJ2000'].append(src_position.dec.value)
    output['Redshift'].append(absorption.redshift.value)
    output['Amplitude'].append(specmodel.amplitude.value)
    output['Luminosity'].append(luminosity[i])
    output['Offset'].append(offset[i])
    output['Index'].append(specmodel.index.value)
    output['Site'].append(sites)
    output['Irf_Zen_pointing'].append(int(IRF_zen[i]))
    output['Irf_Time'].append(float(irftime.split('h')[0]))
    output['Ethreshold'].append(energy_threshold.value)
    output['AboveEthFlag'].append(AboveEthFlag)
    output['N_sims'].append(n_sims)
    output['N_failed'].append(n_failed)
    output['Significance'].append(np.median(significance))
    output['eref0'].append(eref0)
    output['eref1'].append(eref1)
    output['eref2'].append(eref2)
    output['eref3'].append(eref3)
    output['eref4'].append(eref4)
    output['eref5'].append(eref5)
    output['eref6'].append(eref6)
    output['eref7'].append(eref7)
    output['eref8'].append(eref8)
    output['eref9'].append(eref9)
    output['eref10'].append(eref10)
    output['eref11'].append(eref11)
    output['emin0'].append(emin0)
    output['emin1'].append(emin1)
    output['emin2'].append(emin2)
    output['emin3'].append(emin3)
    output['emin4'].append(emin4)
    output['emin5'].append(emin5)
    output['emin6'].append(emin6)
    output['emin7'].append(emin7)
    output['emin8'].append(emin8)
    output['emin9'].append(emin9)
    output['emin10'].append(emin10)
    output['emin11'].append(emin11)
    output['emax0'].append(emax0)
    output['emax1'].append(emax1)
    output['emax2'].append(emax2)
    output['emax3'].append(emax3)
    output['emax4'].append(emax4)
    output['emax5'].append(emax5)
    output['emax6'].append(emax6)
    output['emax7'].append(emax7)
    output['emax8'].append(emax8)
    output['emax9'].append(emax9)
    output['emax10'].append(emax10)
    output['emax11'].append(emax11)
    output['dnde0'].append(dnde0)
    output['dnde1'].append(dnde1)
    output['dnde2'].append(dnde2)
    output['dnde3'].append(dnde3)
    output['dnde4'].append(dnde4)
    output['dnde5'].append(dnde5)
    output['dnde6'].append(dnde6)
    output['dnde7'].append(dnde7)
    output['dnde8'].append(dnde8)
    output['dnde9'].append(dnde9)
    output['dnde10'].append(dnde10)
    output['dnde11'].append(dnde11)
    output['dnde_err0'].append(dnde_err0)
    output['dnde_err1'].append(dnde_err1)
    output['dnde_err2'].append(dnde_err2)
    output['dnde_err3'].append(dnde_err3)
    output['dnde_err4'].append(dnde_err4)
    output['dnde_err5'].append(dnde_err5)
    output['dnde_err6'].append(dnde_err6)
    output['dnde_err7'].append(dnde_err7)
    output['dnde_err8'].append(dnde_err8)
    output['dnde_err9'].append(dnde_err9)
    output['dnde_err10'].append(dnde_err10)
    output['dnde_err11'].append(dnde_err11)
    output['amplitude_fit'].append(amplitude_fit)
    output['amplitude_err_fit'].append(amplitude_err_fit)
    output['index_fit'].append(index_fit)
    output['index_err_fit'].append(index_err_fit)
    output['reference_fit'].append(reference_fit)


Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)


#################### source_name = 402592.0 ####################
#################### n_sims = 1 ####################
D MnSeedGenerator 2 free parameters, FCN pointer 0x7ffefe530960
D InitialGradientCalculator Calculating initial gradient at point 
      1.754592397
       1.22683766
D InitialGradientCalculator Computed initial gradient for parameter par_000_index value 1.75459 [ -1 , 1 ] dirin 1 grd 2 g2 2
D InitialGradientCalculator Computed initial gradient for parameter par_001_amplitude value 1.22684 [ -1 , 1 ] dirin 1 grd 2 g2 2
D Numerical2PGradientCalculator Assumed precision eps 8.88178e-16 eps2 5.96046e-08
D Numerical2PGradientCalculator Calculating gradient around value 4.56468e+07 at point 
      1.754592397
       1.22683766
D Numerical2PGradientCalculator  parameter cycle              x           step             f1             f2            grd             g2

D Numerical2PGradientCalculator par_000_index    0  1.754592396988 1 36950966.57872 95091475.44662 -29070254.433

D MnLineSearch flast 2.49895e+10 f0 1.27561e+07 flast-f0 2.49767e+10 slam 0.00398589
D MnLineSearch denom 3.77246e+15
D MnLineSearch new slam 0.000331888
D MnLineSearch flast 2.01461e+09 f0 1.27561e+07 flast-f0 2.00186e+09 slam 0.000331888
D MnLineSearch denom 4.38928e+16
D MnLineSearch new slam 2.85248e-05
D MnLineSearch flast 1.14736e+08 f0 1.27561e+07 flast-f0 1.0198e+08 slam 2.85248e-05
D MnLineSearch denom 3.38454e+17
D MnLineSearch new slam 3.69928e-06
D MnLineSearch after initial 2-point iter: 
  x0, x1, x2: 0 2.85248e-05 3.69928e-06 
  f0, f1, f2: 1.27561e+07 1.14736e+08 8.12447e+06
D MnLineSearch Iteration 7 
  x0, x1, x2: 0 2.85248e-05 3.69928e-06 
  f0, f1, f2: 1.27561e+07 1.14736e+08 8.12447e+06 
  slamax    : 2.70986e-05 
  p2-p0,p1  : -4.63158e+06 -1.06611e+08 
  a, b, c   : 1.94443e+17 -1.97132e+12 1.27561e+07
D MnLineSearch slam 5.06915e-06 undral -100 overal 2.70986e-05
D MnLineSearch iterate on f3- slam 7 slam 5.06915e-06 xvmin 3.69928e-06
D VariableMetricBuilder Resu

D Numerical2PGradientCalculator par_001_amplitude    0  0.00196090105208 0.0001094644117149 74039.45086419 65965.06073309 36881348.03176 167425697.5601

D Numerical2PGradientCalculator par_001_amplitude    1  0.00196090105208 1.411984667058e-05 70522.03303877 69480.5057169 36881679.60222 167174953.9889

D Numerical2PGradientCalculator 
     Parameter      Gradient           g2           step
 par_000_index -31529.58699513 50971.26522045 0.0008092412315407
par_001_amplitude 36881679.60222 167174953.9889 1.411984667058e-05

D DavidonErrorUpdator 
dx 
  2.559320633e-06
     0.0208433746 
dg 
      1487530.026
        222770399 
delgam 4.64329e+06 gvg 5.2801e+07
D VariableMetricBuilder Updated new point: 
  Parameter: 
      1.754554688
   0.001960901052 
  Gradient: 
       -31529.587
       36881679.6 
  InvHessian: 
  3.3827954e-12  3.8888258e-16
  3.8888258e-16  1.8712876e-10 
  Edm: 723632
I VariableMetricBuilder    5 - FCN =       70001.25271 Edm =       723632.4338 NCalls =     73
D

D MnLineSearch after initial 2-point iter: 
  x0, x1, x2: 0 1 0.650929 
  f0, f1, f2: 14.1866 14.1706 14.1641
D MnLineSearch Iteration 3 
  x0, x1, x2: 0 1 0.650929 
  f0, f1, f2: 14.1866 14.1706 14.1641 
  slamax    : 5 
  p2-p0,p1  : -0.0224696 -0.00651279 
  a, b, c   : 0.0531767 -0.0691335 14.1866
D MnLineSearch slam 0.650035 undral -100 overal 1000
D MnLineSearch iterate on f3- slam 3 slam 0.650035 xvmin 0.650929
D VariableMetricBuilder Result after line search : 
  x = 0.650929 
  Old Fval = 14.1866 
  New Fval = 14.1641 
  NFcalls = 118
D Numerical2PGradientCalculator Assumed precision eps 8.88178e-16 eps2 5.96046e-08
D Numerical2PGradientCalculator Calculating gradient around value 14.1641 at point 
      1.754554715
 -1.049887943e-06
D Numerical2PGradientCalculator  parameter cycle              x           step             f1             f2            grd             g2

D Numerical2PGradientCalculator par_000_index    0  1.754554715329 0.002179336931282 14.16847106628 14.1597

...

D MnLineSearch flast 10.5907 f0 100.335 flast-f0 -89.7447 slam 1
D MnLineSearch denom 167.343
D MnLineSearch new slam 1.03629
D VariableMetricBuilder Result after line search : 
  x = 1 
  Old Fval = 100.335 
  New Fval = 10.5907 
  NFcalls = 107
D Numerical2PGradientCalculator Assumed precision eps 8.88178e-16 eps2 5.96046e-08
D Numerical2PGradientCalculator Calculating gradient around value 10.5907 at point 
      1.686292079
 -1.059252756e-05
D Numerical2PGradientCalculator  parameter cycle              x           step             f1             f2            grd             g2

D Numerical2PGradientCalculator par_000_index    0  1.686292079274 0.000242544121366 10.59095909191 10.59043700111 1.07628006581 1.199774981603

D Numerical2PGradientCalculator par_000_index    1  1.686292079274 0.002146295306091 10.59301079151 10.58839075794 1.076280965323 1.199807417425

D Numerical2PGradientCalculator par_001_amplitude    0  -1.059252756351e-05 1.796425057266e-08 10.58770687077 10.593692

D Numerical2PGradientCalculator par_000_index    1  2.536895836899 0.003443534957699 8.925247441694 8.928821027976 -0.5188834040668 0.3991841737598

D Numerical2PGradientCalculator par_001_amplitude    0  9.718573653661e-06 2.146316703903e-08 8.927931448255 8.92614104981 41708.62672488 19019997023.43

D Numerical2PGradientCalculator par_001_amplitude    1  9.718573653661e-06 1.577573241522e-08 8.927692219059 8.926376250688 41708.62995183 19019991525.81

D Numerical2PGradientCalculator 
     Parameter      Gradient           g2           step
 par_000_index -0.5188834040668 0.3991841737598 0.003443534957699
par_001_amplitude 41708.62995183 19019991525.81 1.577573241522e-08

D DavidonErrorUpdator 
dx 
     0.8506037565
  4.655912801e-06 
dg 
    -0.1787564867
      41799.62074 
delgam 0.0425645 gvg 0.182923
D VariableMetricBuilder Updated new point: 
  Parameter: 
      2.536895837
  9.718573654e-06 
  Gradient: 
    -0.5188834041
      41708.62995 
  InvHessian: 
      43.644381  0.0002

D Numerical2PGradientCalculator  parameter cycle              x           step             f1             f2            grd             g2

D Numerical2PGradientCalculator par_000_index    0  3.79231710532 0.002579438147314 8.385222204343 8.387926541652 -0.5242105361743 0.6565736976863

D Numerical2PGradientCalculator par_001_amplitude    0  3.278425594325e-06 9.270364735498e-09 8.385526793883 8.387628242904 -113342.3053707 124032441764

D Numerical2PGradientCalculator par_001_amplitude    1  3.278425594325e-06 6.007183674001e-09 8.385893558705 8.387255294635 -113342.2918033 124032252061.5

D Numerical2PGradientCalculator 
     Parameter      Gradient           g2           step
 par_000_index -0.5242105361743 0.6565736976863 0.002579438147314
par_001_amplitude -113342.2918033 124032252061.5 6.007183674001e-09

D DavidonErrorUpdator 
dx 
     0.5064566265
 -2.238565591e-06 
dg 
    -0.0444259313
     -90274.30088 
delgam 0.179585 gvg 0.475593
D VariableMetricBuilder Updated new point: 

D MnHesse cycle 0 mul 0 	sag = 2.19954e-06 d = 1.94063e-09
D MnHesse g1 = -359.646 g2 = 1.16809e+12 step = 1.94063e-09 d = 6.21001e-08 diffd = 0.96875 diffg2 = 0
D MnHesse Second derivatives 
      3.015439689
  1.168090489e+12
D HessianGradientCalculator HGC Param : 0 	 new g1 = -0.0026922 gstep = 0.000273437 dgrd = 3.23405e-06
D HessianGradientCalculator HGC Param : 1 	 new g1 = -359.612 gstep = 3.88126e-10 dgrd = 0.0332451
D MnHesse Original error matrix 
      3.0154397      1849826.1
      1849826.1  1.1680905e+12
D MnHesse PosDef error matrix 
      3.0154397      1849826.1
      1849826.1  1.1680905e+12
D MnHesse Hessian is ACCURATE. New state: 
  First derivative: 
  -0.002692199477
     -359.6123599 
  Second derivative: 
      3.015439689
  1.168090489e+12 
  Gradient step: 
  0.0002734366296
  3.881257516e-10 
  Covariance matrix: 
      11.628602 -1.8415432e-05
 -1.8415432e-05  3.0019376e-11 
  Edm: 2.62539e-05
I VariableMetricBuilder After Hessian
I VariableMetricBuilder  

0.8868077627060499


In [18]:
# Save renortets
renortets = Table()
renortets['Source_Name'] = Column(output['Source_Name'], description='Source name')
renortets['RA_pointing'] = Column(output['RA_pointing'], format='{:.6f}', unit='degree', description='Right Ascension')
renortets['DEC_pointing'] = Column(output['DEC_pointing'], format='{:.6f}', unit='degree', description='Declination')
renortets['RAJ2000'] = Column(output['RAJ2000'], format='{:.6f}', unit='degree', description='Right Ascension')
renortets['DEJ2000'] = Column(output['DEJ2000'], format='{:.6f}', unit='degree', description='Declination')
renortets['Redshift'] = Column(output['Redshift'], format='{:.4f}', unit=None, description='Redshift')
renortets['Amplitude'] = Column(output['Amplitude'], format='{:.4f}', unit=None, description='Amplitude')
renortets['Index'] = Column(output['Index'], format='{:.4f}', unit=None, description='Index')
renortets['Luminosity'] = Column(output['Luminosity'], format='{:.4f}', unit=None, description='Luminosity')
renortets['Offset'] = Column(output['Offset'], format='{:.6f}', unit=None, description='Offset')
renortets['Site'] = Column(output['Site'], description='Site of the considered CTA array')
renortets['Ethreshold'] = Column(output['Ethreshold'], unit='TeV', description='Effective energy threshold')
renortets['EthFlag'] = Column(output['AboveEthFlag'], unit=None, description='Is requested minimal energy high enough, i.e. above energy threshold from IRF ?')
renortets['N_sims'] = Column(output['N_sims'], unit=None, description='Number of successfully thrown simulations')
renortets['N_failed'] = Column(output['N_failed'], unit=None, description='Number of failed simulations')
renortets['Irf_Zen_pointing'] = Column(output['Irf_Zen_pointing'], unit='deg', description='Irf_Zen_pointing')
renortets['Significance'] = Column(output['Significance'], format='{:.2f}', unit='', description='Median significance')
renortets['eref0'] = Column(output['eref0'], format='{:.2f}', unit='', description='eref')
renortets['eref1'] = Column(output['eref1'], format='{:.2f}', unit='', description='eref')
renortets['eref2'] = Column(output['eref2'], format='{:.2f}', unit='', description='eref')
renortets['eref3'] = Column(output['eref3'], format='{:.2f}', unit='', description='eref')
renortets['eref4'] = Column(output['eref4'], format='{:.2f}', unit='', description='eref')
renortets['eref5'] = Column(output['eref5'], format='{:.2f}', unit='', description='eref')
renortets['eref6'] = Column(output['eref6'], format='{:.2f}', unit='', description='eref')
renortets['eref7'] = Column(output['eref7'], format='{:.2f}', unit='', description='eref')
renortets['eref8'] = Column(output['eref8'], format='{:.2f}', unit='', description='eref')
renortets['eref9'] = Column(output['eref9'], format='{:.2f}', unit='', description='eref')
renortets['eref10'] = Column(output['eref10'], format='{:.2f}', unit='', description='eref')
renortets['eref11'] = Column(output['eref11'], format='{:.2f}', unit='', description='eref')
renortets['emax0'] = Column(output['emax0'], format='{:.2f}', unit='', description='emax')
renortets['emax1'] = Column(output['emax1'], format='{:.2f}', unit='', description='emax')
renortets['emax2'] = Column(output['emax2'], format='{:.2f}', unit='', description='emax')
renortets['emax3'] = Column(output['emax3'], format='{:.2f}', unit='', description='emax')
renortets['emax4'] = Column(output['emax4'], format='{:.2f}', unit='', description='emax')
renortets['emax5'] = Column(output['emax5'], format='{:.2f}', unit='', description='emax')
renortets['emax6'] = Column(output['emax6'], format='{:.2f}', unit='', description='emax')
renortets['emax7'] = Column(output['emax7'], format='{:.2f}', unit='', description='emax')
renortets['emax8'] = Column(output['emax8'], format='{:.2f}', unit='', description='emax')
renortets['emax9'] = Column(output['emax9'], format='{:.2f}', unit='', description='emax')
renortets['emax10'] = Column(output['emax10'], format='{:.2f}', unit='', description='emax')
renortets['emax11'] = Column(output['emax11'], format='{:.2f}', unit='', description='emax')
renortets['emin0'] = Column(output['emin0'], format='{:.2f}', unit='', description='emin')
renortets['emin1'] = Column(output['emin1'], format='{:.2f}', unit='', description='emin')
renortets['emin2'] = Column(output['emin2'], format='{:.2f}', unit='', description='emin')
renortets['emin3'] = Column(output['emin3'], format='{:.2f}', unit='', description='emin')
renortets['emin4'] = Column(output['emin4'], format='{:.2f}', unit='', description='emin')
renortets['emin5'] = Column(output['emin5'], format='{:.2f}', unit='', description='emin')
renortets['emin6'] = Column(output['emin6'], format='{:.2f}', unit='', description='emin')
renortets['emin7'] = Column(output['emin7'], format='{:.2f}', unit='', description='emin')
renortets['emin8'] = Column(output['emin8'], format='{:.2f}', unit='', description='emin')
renortets['emin9'] = Column(output['emin9'], format='{:.2f}', unit='', description='emin')
renortets['emin10'] = Column(output['emin10'], format='{:.2f}', unit='', description='emin')
renortets['emin11'] = Column(output['emin11'], format='{:.2f}', unit='', description='emin')
renortets['dnde0'] = Column(output['dnde0'], format='{:.2f}', unit='', description='dnde')
renortets['dnde1'] = Column(output['dnde1'], format='{:.2f}', unit='', description='dnde')
renortets['dnde2'] = Column(output['dnde2'], format='{:.2f}', unit='', description='dnde')
renortets['dnde3'] = Column(output['dnde3'], format='{:.2f}', unit='', description='dnde')
renortets['dnde4'] = Column(output['dnde4'], format='{:.2f}', unit='', description='dnde')
renortets['dnde5'] = Column(output['dnde5'], format='{:.2f}', unit='', description='dnde')
renortets['dnde6'] = Column(output['dnde6'], format='{:.2f}', unit='', description='dnde')
renortets['dnde7'] = Column(output['dnde7'], format='{:.2f}', unit='', description='dnde')
renortets['dnde8'] = Column(output['dnde8'], format='{:.2f}', unit='', description='dnde')
renortets['dnde9'] = Column(output['dnde9'], format='{:.2f}', unit='', description='dnde')
renortets['dnde10'] = Column(output['dnde10'], format='{:.2f}', unit='', description='dnde')
renortets['dnde11'] = Column(output['dnde11'], format='{:.2f}', unit='', description='dnde')
renortets['dnde_err0'] = Column(output['dnde_err0'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err1'] = Column(output['dnde_err1'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err2'] = Column(output['dnde_err2'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err3'] = Column(output['dnde_err3'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err4'] = Column(output['dnde_err4'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err5'] = Column(output['dnde_err5'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err6'] = Column(output['dnde_err6'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err7'] = Column(output['dnde_err7'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err8'] = Column(output['dnde_err8'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err9'] = Column(output['dnde_err9'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err10'] = Column(output['dnde_err10'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err11'] = Column(output['dnde_err11'], format='{:.2f}', unit='', description='dnde_err')
renortets['amplitude_fit'] = Column(output['amplitude_fit'], format='{:.2f}', unit='', description='amplitude_fit')
renortets['amplitude_err_fit'] = Column(output['amplitude_err_fit'], format='{:.2f}', unit='', description='amplitude_err_fit')
renortets['index_fit'] = Column(output['index_fit'], format='{:.2f}', unit='', description='index_fit')
renortets['index_err_fit'] = Column(output['index_err_fit'], format='{:.2f}', unit='', description='index_err_fit')
renortets['reference_fit'] = Column(output['reference_fit'], format='{:.2f}', unit='', description='reference')

In [19]:
renortets

Source_Name,RA_pointing,DEC_pointing,RAJ2000,DEJ2000,Redshift,Amplitude,Index,Luminosity,Offset,Site [1],Ethreshold,EthFlag,N_sims,N_failed,Irf_Zen_pointing,Significance,eref0 [1],eref1 [1],eref2 [1],eref3 [1],eref4 [1],eref5 [1],eref6 [1],eref7 [1],eref8 [1],eref9 [1],eref10 [1],eref11 [1],emax0 [1],emax1 [1],emax2 [1],emax3 [1],emax4 [1],emax5 [1],emax6 [1],emax7 [1],emax8 [1],emax9 [1],emax10 [1],emax11 [1],emin0 [1],emin1 [1],emin2 [1],emin3 [1],emin4 [1],emin5 [1],emin6 [1],emin7 [1],emin8 [1],emin9 [1],emin10 [1],emin11 [1],"dnde0 [1,1,1]","dnde1 [1,1,1]","dnde2 [1,1,1]","dnde3 [1,1,1]","dnde4 [1,1,1]","dnde5 [1,1,1]","dnde6 [1,1,1]","dnde7 [1,1,1]","dnde8 [1,1,1]","dnde9 [1,1,1]","dnde10 [1,1,1]","dnde11 [1,1,1]","dnde_err0 [1,1,1]","dnde_err1 [1,1,1]","dnde_err2 [1,1,1]","dnde_err3 [1,1,1]","dnde_err4 [1,1,1]","dnde_err5 [1,1,1]","dnde_err6 [1,1,1]","dnde_err7 [1,1,1]","dnde_err8 [1,1,1]","dnde_err9 [1,1,1]","dnde_err10 [1,1,1]","dnde_err11 [1,1,1]",amplitude_fit [1],amplitude_err_fit [1],index_fit [1],index_err_fit [1],reference_fit [1]
,deg,deg,deg,deg,,,,,,,TeV,,,,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,bool,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
402592,251.315112,10.620350,248.240700,9.164700,0.1919,0.0000,1.8182,40.8288,3.360259,North,0.10000000000000006,True,1,0,20,-0.37,0.13,0.20,0.32,0.50,0.79,1.26,2.00,3.16,5.01,7.94,12.59,19.95,0.16,0.25,0.40,0.63,1.00,1.58,2.51,3.98,6.31,10.00,15.85,25.12,0.10,0.16,0.25,0.40,0.63,1.00,1.58,2.51,3.98,6.31,10.00,15.85,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.71,0.61,0.20
628502,299.994430,43.648297,301.710900,41.603800,0.4461,0.0000,1.8338,40.8478,2.402995,North,0.06309573444801937,True,1,0,20,0.89,0.13,0.20,0.32,0.50,0.79,1.26,2.00,3.16,5.01,7.94,12.59,19.95,0.16,0.25,0.40,0.63,1.00,1.58,2.51,3.98,6.31,10.00,15.85,25.12,0.10,0.16,0.25,0.40,0.63,1.00,1.58,2.51,3.98,6.31,10.00,15.85,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.90,4.82,0.20


In [22]:
file = open("/home/bornagain/Documents/GitHub/gammapy/gammapy-notebooks/0.20.1/presentations/gammapy/results/North_PL_D11_1x.txt","+w")
file.write('Source_Name  RA  DEC  Redshift  Luminosity  Index  Amplitude  E_th  Significance  \r\n')

for i in range(len(renortets['RAJ2000'])):
    file.write(str(int(source_name[i])) + '  ' + str(renortets['RAJ2000'][i]) + '  ' + str(renortets['DEJ2000'][i]) + '  ' + str(renortets['Redshift'][i]) + '  ' + str(renortets['Luminosity'][i]) + '  ' + str(renortets['Index'][i]) + '  ' + str(renortets['Amplitude'][i]) + '  ' + str(renortets['Ethreshold'][i]) + '  ' + str(renortets['Significance'][i]) + '\n')

In [23]:
df_result = pd.read_csv('results/North_PL_D11_1x.txt', sep='  ', engine='python')
display(df_result)

,Source_Name,RA,DEC,Redshift,Luminosity,Index,Amplitude,E_th,Significance
0,402592,248.2407,9.1647,0.1919,40.8288,1.8182,1.298100e-11,0.100000,-0.369900
1,628502,301.7109,41.6038,0.4461,40.8478,1.8338,2.293000e-12,0.063096,0.886808


In [ ]:
display(selected_obs)

___

🔝 [Back to Top](#intro)<br>